In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import csv
import requests
import pandas as pd
import mysql.connector
from mysql.connector import errorcode
import re # necesario para la siguiente línea de código: bio_limpia = re.sub('<[^<]+?>', '', bio_content).strip() if bio_content else '' (quita las etiquetas HTML)

In [4]:
# Spotify
CLIENT_ID = '9189e74191a848bf82c60efbf06229c6'
CLIENT_SECRET = '36c5b1edfa3744e7ac992361fa2b5b02'

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
))

generos = ['pop', 'metal', 'reggaeton', 'rap']
años = [2015, 2016]

datos_spotify = []

for genero in generos:
    for ano in años:
        for offset in range(0, 100, 50):
            resultados = sp.search(q=f'genre:"{genero}" year:{ano}',
                                   type='track',
                                   limit=50,
                                   offset=offset)
            for cancion in resultados['tracks']['items']:
                artistas = [artista['name'] for artista in cancion['artists']]
                datos_spotify.append({
                    'artistas': ', '.join(artistas), # el campo de artistas en nuestro CSV será una cadena de texto (artista1, artista2, etc.)
                    'colaboracion': len(artistas) > 1, # este código crea un booleano que devuelve valor TRUE si la canción es una colaboración entre dos o más artistas y FALSE si solo hay un artista. 
                    'genero_musical': genero,
                    'nombre': cancion['name'],
                    'ano_lanzamiento': cancion['album']['release_date'][:4],
                    'id': cancion['id']
                })

with open('spotify_datos.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['artistas', 'colaboracion', 'genero_musical', 'nombre', 'ano_lanzamiento', 'id'])
    writer.writeheader()
    writer.writerows(datos_spotify)

print(f"✅ Guardado: spotify_datos.csv ({len(datos_spotify)} canciones)")

# Last.fm
API_KEY_LASTFM = '472626d71b58f9fbdd11821189cb8716'
BASE_URL = "http://ws.audioscrobbler.com/2.0/"

df = pd.read_csv('spotify_datos.csv')
todos_artistas = []
for lista_artistas in df['artistas']:
    artistas = [art.strip() for art in lista_artistas.split(',') if art.strip()]
    todos_artistas.extend(artistas)

artistas_unicos = sorted(set(todos_artistas)) # elimina duplicados

print(f"Artistas únicos detectados: {len(artistas_unicos)}")

datos_lastfm = []

for artista in artistas_unicos:
    params = {
        "method": "artist.getInfo",
        "artist": artista,
        "api_key": API_KEY_LASTFM,
        "format": "json",
        "lang": "es"
    }
    resp = requests.get(BASE_URL, params=params).json()
    if "artist" in resp:
        bio_content = resp['artist'].get('bio', {}).get('content', '')
        bio_limpia = re.sub('<[^<]+?>', '', bio_content).strip() if bio_content else ''
        stats = resp['artist'].get('stats', {})
        similares = resp['artist'].get('similar', {}).get('artist', [])
        similares_nombres = [sim.get('name', '') for sim in similares]
        datos_lastfm.append({
            'artist': artista,
            'biografia': bio_limpia,
            'playcount': stats.get('playcount', 0),
            'oyentes': stats.get('listeners', 0),
            'similares': ', '.join(similares_nombres),
        })

with open('lastfm_datos.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['artist', 'biografia', 'playcount', 'oyentes', 'similares'])
    writer.writeheader()
    writer.writerows(datos_lastfm)

print(f"✅ Guardado: lastfm_datos.csv ({len(datos_lastfm)} artistas)")

✅ Guardado: spotify_datos.csv (800 canciones)
Artistas únicos detectados: 826
✅ Guardado: lastfm_datos.csv (794 artistas)


In [3]:
# Conexión Python-MySQL.
cnx = mysql.connector.connect(
    user='root',
    password='AlumnaAdalab',
    host='127.0.0.1',
    database='proyecto_grupal_mod_2'
)

print(cnx) # Muestra la conexión.

mycursor = cnx.cursor()
mycursor.execute("SELECT DATABASE();")
db_actual = mycursor.fetchone()
print("Base de datos activa:", db_actual)

Base de datos activa: ('proyecto_grupal_mod_2',)


In [4]:
df_spotify = pd.read_csv('spotify_datos.csv')
df_lastfm = pd.read_csv('lastfm_datos.csv')

In [5]:
# Iniciamos los cursores. Usamos dos para evitar errores del tipo "Unread result found" a la hora de ejecutar en Python.
mycursor = cnx.cursor()
mycursor_bucle = cnx.cursor()
# Lastfm_datos
with open('lastfm_datos.csv', 'r', encoding='utf-8') as archivo_csv:
    reader = csv.DictReader(archivo_csv)
    for fila in reader:
        # ON DUPLICATE KEY UPDATE sirve para manejar duplicados dentro de tabla artistas, de cara a futuras importaciones de datos en MYSQL (en esta no hay duplicados, dado que hemos usado una lsita de artistas únicos.)
        sql = """
            INSERT INTO artistas (nombre, biografia, playcount, oyentes, similares)
            VALUES (%s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE 
                biografia = VALUES(biografia),
                playcount = VALUES(playcount),
                oyentes = VALUES(oyentes),
                similares = VALUES(similares)
        """
    
        valores = (
            fila['artist'],
            fila['biografia'],
            int(fila['playcount']),
            int(fila['oyentes']),
            fila.get('similares', '')
        )
        mycursor.execute(sql, valores)

print("Datos insertados/actualizados en `artistas` correctamente")

# Spotify_datos
with open('spotify_datos.csv', 'r', encoding='utf-8') as archivo_csv:
    reader = csv.DictReader(archivo_csv)
    for fila in reader:
        sql_cancion = """
            INSERT INTO canciones (nombre, genero_musical, tipo, año_lanzamiento, id_spotify)
            VALUES (%s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE nombre = VALUES(nombre)
        """
        valores_cancion = (
            fila['nombre'],
            fila['genero_musical'],
            fila['genero_musical'],
            int(fila['ano_lanzamiento']),
            fila['id']
        )
        mycursor.execute(sql_cancion, valores_cancion)

        # Ahora obtenemos el id de cada canción
        mycursor_bucle.execute("SELECT id_cancion FROM canciones WHERE id_spotify = %s", (fila['id'],))
        id_cancion = mycursor_bucle.fetchone()[0]

        # Ahora insertamos la relación en la tabla intermedia por cada artista
        artistas = [art.strip() for art in fila['artistas'].split(',')]

        for artista in artistas:
            mycursor_bucle.execute("SELECT id_artista FROM artistas WHERE nombre = %s", (artista,))
            res = mycursor_bucle.fetchone()
            if res:
                id_artista = res[0]
                sql_relacion = """
                    INSERT INTO canciones_artistas (id_cancion, id_artista)
                    VALUES (%s, %s)
                    ON DUPLICATE KEY UPDATE id_cancion = id_cancion
                """
                mycursor_bucle.execute(sql_relacion, (id_cancion, id_artista))

print("Datos insertados en `canciones` correctamente")

cnx.commit()

# Guardar cambios y cerrar
# cnx.commit()
mycursor.close()
mycursor_bucle.close()
print("Todos los datos han sido insertados.")


Datos insertados/actualizados en `artistas` correctamente
Datos insertados en `canciones` correctamente
Todos los datos han sido insertados.
